In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

In [2]:
def haunted_scrap(z):  
    
    mp_state = ["California1", "California2", "California3", "Texas1", "Texas2"]
    
    file_name = z
    zlower = z.lower()
    
    if z in mp_state:
        z = z[:-1]
    else:
        pass
  
    # import url and parse link
    
    url = 'http://www.theshadowlands.net/places/'+ zlower +'.htm'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # create new empty list
    x = []
    
    # get text and add to the empty list
    x = soup.get_text()
    
    # split text by '\n\n'
    x = x.rsplit('\n\n')
    
    # the new empty list below will hold the previous text and split again using '-'
    x_list = []

    for line in x:
        name = line.split('-')
        x_list.append(name)
        
    # return x_list (test)
    
    # these 3 empty lists will hold the clean text and will be converted to the dataframe columns
    
    x_city = []
    x_venue = []
    x_venue_add = []
    
    # the next 3 for loops will fill the 3 lists
    # city
    for item in x_list:
          
        try:
            item = item[0]
            x_city.append(item.title().strip())

        except IndexError:
            x_city.append("null")
    
    # venue
    for item in x_list:
    
        try:
            item = item[1]
            x_venue.append(item.title().strip())

        except IndexError:
            x_venue.append("null")
    

    # run lines below to make sure all hgave the same lenght
    
    # print(len(x_city))
    # print(len(x_venue))
        
    # print(len(new_venue_add))
    
    # append all lists to a new state dataframe
    
    x_df = pd.DataFrame(
        {'State': z.title().strip(),
         'County': x_city,
         'City': x_city,
         'Venue': x_venue,
         'Latitude': " ",
         'Longitude': " ",
        })
    
    # drop null values - venue column. 
    indexNames = x_df[x_df["Venue"] == "null"].index

    x_df.drop(indexNames, inplace = True)

    x_df.drop(x_df.index[0], inplace = True)
    
    # RUNS ONLY FOR TEXAS AND CALIFORNIA:
    
    if file_name == "Texas2":
        x_df.drop(x_df.index[0:2], inplace = True)
        
    if file_name == "California1":
        x_df.drop(x_df.index[0:3], inplace = True)
        
    if file_name == "California2":
        x_df.drop(x_df.index[0:1], inplace = True)
        
    if file_name == "California3":
        x_df.drop(x_df.index[0:2], inplace = True)
        
    # replace '\n' to ' '
    cols_to_check = ['County', 'City', 'Venue']

    x_df[cols_to_check] = x_df[cols_to_check].replace({'\n':' ', '<':'', '>':'' }, regex=True)
    
    
    
    # THE NEXT FEW LINE WILL FILTER AND IMPORT THE COUNTY NAME FROM THE DF (county_data.csv) - see 2nd line on the top
    
    county_data = pd.read_csv("us_counties.csv")
    
    county_data["County"] = county_data["County"].str.title()
    
    county_data = county_data.rename(columns={"State full": "State"})
    
    county_data = county_data[(county_data["State"] == z)]
    
    # create a dictionary only with the current state counties.
    
    di = dict(zip(county_data.City, county_data.County))
    
    #replace on the dataframe using .replace()
    
    x_df = x_df.replace({"County": di})
 

    # COLLECT LAT LONG:
    
    # reset the index, so the Lat/Long can match the correct Cities
    x_df = x_df.reset_index()
    x_df = x_df.drop('index', axis = 1)
    
    
    # GETTING THE GEO COORDINATES
    
    # DON'T FORGET TO IMPORT THE CORRECT LIBRARIES:
   
    # from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
    # import geocoder # to get coordinates
    
    state_name = x_df['State']
    state_name = state_name.values[0]
    
    # function to get coordinates
    def get_latlng(city):
        # initialize your variable to None
        lat_lng_coords = None
        # loop until you get the coordinates
        while(lat_lng_coords is None):
            g = geocoder.arcgis(city + ',' + state_name)
            lat_lng_coords = g.latlng
        return lat_lng_coords
    
    # call the function to get the coordinates, store in a new list using list comprehension
    coords = [ get_latlng(city) for city in x_df['City'] ]
    
    # create temporary dataframe to populate the coordinates into Latitude and Longitude
    df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
    
    # merge the coordinates into the original dataframe
    x_df['Latitude'] = df_coords['Latitude']
    x_df['Longitude'] = df_coords['Longitude']

    # PRINT DATAFRAME HEAD
    #print(url)
    #return x_df.head(5)
    
    # SAVE TO CSV FILES
    x_df.to_csv(file_name + '_df.csv')

In [103]:
#run function in one state:

#haunted_scrap(z = 'Alabama')

In [6]:
#list of states withou spaces. California has 3 pages and Texas 2.

state_names = ["Alaska", "Alabama", "Arkansas", "Arizona", "California1", "California2", "California3", "Colorado", "Connecticut", "DC", 
               "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", 
               "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", 
               "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", 
               "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", 
               "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas1", "Texas2", "Utah", "Virginia", 
               "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

In [7]:
for state in state_names:
    haunted_scrap(z = state)

In [ ]:
#put together all tables:

mycsvdir = '/Users/"your_user_name_here"/Documents/Data Science/Untitled Folder/CSV State Files/'

csvfiles = glob.glob(os.path.join(mycsvdir, '*.csv'))

dataframes = []

for csvfile in csvfiles:
    df = pd.read_csv(csvfile)
    dataframes.append(df)

haunted_df = pd.concat(dataframes, ignore_index = True)

haunted_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

haunted_df['State'].replace({'Dc': 'District of Columbia'}, inplace = True)


haunted_df.to_csv('haunted_df.csv', index=False)

haunted_df.head()